In [1]:
import serial
import time
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from std_msgs.msg import Float64
from dynamixel_sdk import *
import datetime
import threading
import serial
import re
import pandas as pd
import glob

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

/tmp/ipykernel_125954/2284580335.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

<IPython.core.display.Javascript object>

In [4]:
class Dynamixel_motor:
    def __init__(self, dxl_id, baudrate, device_name):
        self.dxl_id = dxl_id
        self.baudrate = baudrate
        self.device_name = device_name
        self.protocol_version = 2.0

        # Initialize PortHandler instance
        # Set the port path
        # Get methods and members of PortHandlerLinux or PortHandlerWindows
        self.portHandler = PortHandler(self.device_name)

        # Initialize PacketHandler instance
        # Set the protocol version
        # Get methods and members of Protocol1PacketHandler or Protocol2PacketHandler
        self.packetHandler = PacketHandler(self.protocol_version)

        # Open port
        if self.portHandler.openPort():
            print("Succeeded to open the port")
        else:
            print("Failed to open the port")
            print("Press any key to terminate...")
            self.close_port()

        # Set port baudrate
        if self.portHandler.setBaudRate(self.baudrate):
            print("Succeeded to change the baudrate")
        else:
            print("Failed to change the baudrate")
            print("Press any key to terminate...")
            self.close_port()

    def __del__(self):
        print('Destructor called.')
        self.portHandler.closePort()

    def close_port(self):
        self.torque_disable()
        self.portHandler.closePort()
        print('closing port.')

    def ping_motor(self):

        # Try to ping the Dynamixel
        # Get Dynamixel model number
        dxl_model_number, dxl_comm_result, dxl_error = self.packetHandler.ping(self.portHandler, self.dxl_id)
        print(f"dxl_model_number is: {dxl_model_number}")
        print(f"dxl_comm_result is: {dxl_comm_result}")
        print(f"dxl_error is: {dxl_error}")
        print(f"COMM_SUCCESS is {COMM_SUCCESS}")
        if dxl_comm_result != COMM_SUCCESS:
            print("statement 0")
            print("%s" % self.packetHandler.getTxRxResult(dxl_comm_result))
            self.close_port()
        elif dxl_error != 0:
            print("statement 1")
            print("%s" % self.packetHandler.getRxPacketError(dxl_error))
            self.close_port()
        else:
            print("statement 2")
            print("[ID:%03d] ping Succeeded. Dynamixel model number : %d" % (self.dxl_id, dxl_model_number))

    def torque_enable(self):
        TORQUE_ENABLE_REG = 64
        TORQUE_ENABLE = 1
        result, error = self.packetHandler.write4ByteTxRx(self.portHandler, self.dxl_id, TORQUE_ENABLE_REG, TORQUE_ENABLE)
        return

    def torque_disable(self):
        TORQUE_ENABLE_REG = 64
        TORQUE_ENABLE = 0
        result, error = self.packetHandler.write4ByteTxRx(self.portHandler, self.dxl_id, TORQUE_ENABLE_REG, TORQUE_ENABLE)
        return

    def read_operation_mode(self):
        OPERATING_MODE_REG = 11
        data, result, error = self.packetHandler.read1ByteTxRx(self.portHandler, self.dxl_id, OPERATING_MODE_REG)
        print(f"data is: {data}")
        print(f"result is: {result}")
        print(f"error is: {error}")
        return data

    def set_current_position_mode(self):
        OPERATING_MODE_REG = 11
        OPERATION_MODE = 5
        result, error = self.packetHandler.write1ByteTxRx(self.portHandler, self.dxl_id, OPERATING_MODE_REG, OPERATION_MODE)
        return

    def set_goal_current(self, GOAL_CURRENT):
        GOAL_CURRENT_REG = 102
        result, error = self.packetHandler.write2ByteTxRx(self.portHandler, self.dxl_id, GOAL_CURRENT_REG, GOAL_CURRENT)
        return

    def read_present_position(self):
        PRESENT_POSITION_REG = 132
        PRESENT_POSITION, result, error = self.packetHandler.read4ByteTxRx(self.portHandler, self.dxl_id, PRESENT_POSITION_REG)
        return PRESENT_POSITION

    def set_goal_position(self, GOAL_POSITION):
        GOAL_POSITION_REG = 116
        result, error = self.packetHandler.write4ByteTxRx(self.portHandler, self.dxl_id, GOAL_POSITION_REG, GOAL_POSITION)
        return

    def read_present_current(self):
        PRESENT_CURRENT_REG = 126
        PRESENT_CURRENT, result, error = self.packetHandler.read2ByteTxRx(self.portHandler, self.dxl_id, PRESENT_CURRENT_REG)
        return PRESENT_CURRENT


In [5]:
class Nodo(object):
    def __init__(self, dxl_motor):
        
        self.motor_position = 200
        
        self.input_thread = threading.Thread(target=self.get_input)
        self.input_thread.daemon = True
        self.input_thread.start()

        self.weight_reading_1 = []
        self.weight_reading_2 = []
        self.pressure_reading_1 = []
        self.pressure_reading_2 = []
        
        self.command = '3'
        self.dxl_motor = dxl_motor
        self.initialise_motor()
        
        self.ser = serial.Serial('/dev/ttyACM0', 57600, timeout=1)
        self.initialise_arduino()
        
        self.move_jaw(self.motor_position)
        
        self.serial_thread = threading.Thread(target=self.read_arduino)
        self.serial_thread.daemon = True
        self.serial_thread.start()
        
    def initialise_arduino(self):
        time.sleep(4) # wait for the arduino to initialise
        print('Arduino initialised.')
        self.ser.flushInput() # clear the serial buffer
    
    def read_arduino(self):
        while True:
            if self.ser.in_waiting > 0 and (self.command == '1' or self.command == '2'): 
#             if self.ser.in_waiting > 0 and self.command == '1': 
                data = self.ser.readline()
                text = re.findall('\d+\.\d+', data.decode("utf-8"))
                self.weight_reading_1.append(float(text[0]))
                self.weight_reading_2.append(float(text[1]))
                self.pressure_reading_1.append(float(text[2]))
                self.pressure_reading_2.append(float(text[3]))
#                 print(text)
                print(data)
                time.sleep(0.025)
            
            
    def initialise_motor(self):
        self.dxl_motor.set_current_position_mode()
        self.dxl_motor.read_operation_mode()
        self.dxl_motor.set_goal_current(50)
        
    def move_jaw(self, position):
        self.dxl_motor.torque_enable()
#         print(f"current position is {self.dxl_motor.read_present_position()}")
        self.dxl_motor.set_goal_position(position)
    
    def disable_jaw(self):
        self.dxl_motor.torque_disable()
    
    def get_input(self):
        while True:
            self.command = input("Enter something: ")
            time.sleep(1)
    
    def save_data(self):
        directory = '/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/Pressure_force_dataset/'
        
        start_number = 0
             
        header = ['weight_reading_1', 'weight_reading_2', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1']
        previous_df = pd.concat([pd.Series(self.weight_reading_1), pd.Series(self.weight_reading_2), pd.Series(self.pressure_reading_1), pd.Series(self.pressure_reading_2)], axis=1, keys=header)
        
        combined_df_set = pd.read_csv(directory + 'Distance_50mm_cycle.csv')
        
        combined_df_set = pd.concat([combined_df_set, previous_df], axis=0)
        
        combined_df_set.to_csv(directory + 'Distance_50mm_cycle.csv', index=False)
        
    def start(self):
        
        while True:
            
            if self.command == '1':
                
                if self.motor_position >= 1900:
                    time.sleep(2)
                    self.command = '2'
                    continue
                
                self.motor_position = self.motor_position + 5
                self.move_jaw(self.motor_position)
                time.sleep(0.01)
                
            
            elif self.command == '2':
                
                if self.motor_position <= 200:
                    self.command = '3'
                    continue
                
#                 self.motor_position = self.motor_position - 20
                self.motor_position = self.motor_position - 5
                self.move_jaw(self.motor_position)
                time.sleep(0.01)
                
            elif self.command == '3':
                self.disable_jaw()
                self.ser.flushInput()
                time.sleep(0.1)
            
            
if __name__ == '__main__':
    dxl_motor = Dynamixel_motor(1, 57600, '/dev/ttyUSB0')
    my_node = Nodo(dxl_motor)
    my_node.start()

Succeeded to open the port
Succeeded to change the baudrate
data is: 5
result is: 0
error is: 0
Arduino initialised.
Enter something: 1
b'0.01,0.02,-0.05,0.11\r\n'
b'0.01,0.01,-0.01,0.48\r\n'
b'0.01,0.01,0.01,0.16\r\n'
b'0.02,0.01,0.02,0.00\r\n'
b'0.01,0.01,0.03,0.42\r\n'
b'0.00,0.01,0.03,0.13\r\n'
b'0.01,0.01,0.03,-0.01\r\n'
b'0.00,0.00,0.03,-0.09\r\n'
b'-0.01,-0.00,0.03,-0.12\r\n'
b'0.00,0.00,0.03,-0.14\r\n'
b'-0.00,-0.00,-0.47,0.35\r\n'
b'-0.00,0.01,-0.22,0.10\r\n'
b'-0.01,0.01,-0.09,-0.03\r\n'
b'-0.01,0.00,-0.03,-0.09\r\n'
b'-0.01,0.01,-0.00,0.37\r\n'
b'-0.01,-0.00,0.02,0.11\r\n'
b'0.01,0.01,-0.48,-0.53\r\n'
b'-0.00,-0.01,-0.72,-0.34\r\n'
b'-0.00,0.01,-0.35,-0.25\r\n'
b'-0.01,-0.01,-0.16,-0.20\r\n'
b'-0.00,-0.00,-1.06,-0.18\r\n'
b'-0.00,0.00,-0.52,-0.17\r\n'
b'-0.01,0.05,-0.74,-0.16\r\n'
b'-0.00,0.46,-0.86,0.34\r\n'
b'0.32,1.37,-0.41,0.09\r\n'
b'1.78,2.70,0.31,0.47\r\n'
b'4.20,4.63,0.67,1.15\r\n'
b'6.76,6.86,1.35,2.00\r\n'
b'9.30,9.08,2.19,2.42\r\n'
b'11.93,11.36,2.61,3.13\r\n'
b'1

KeyboardInterrupt: 

In [6]:
my_node.save_data()